# XArrayDataset

In [2]:
import autoroot
from oceanbench._src.datasets.base import XRDABatcher
import numpy as np
import pandas as pd
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)
from xarray_dataclasses import asdataarray, asdataset
%load_ext autoreload
%autoreload 2


In [3]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, CoordinateAxis,
    TimeAxis, SSH2D, SSH2DT, Bounds
)
from xarray_dataclasses import asdataarray, asdataset, Data, Name, Coordof, Coord
from oceanbench._src.geoprocessing.gridding import create_coord_grid

In [4]:
from dataclasses import dataclass
from typing import Literal, Tuple


X = Literal["x"]
Y = Literal["y"]
Z = Literal["z"]

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable2D:
    data: Data[tuple[X,Y], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    name: Name[str] = "var"


@dataclass
class Variable3D:
    data: Data[tuple[X, Y, Z], np.ndarray]
    x: Coord[X, np.ndarray] = 0
    y: Coord[Y, np.ndarray] = 0
    z: Coord[Z, np.ndarray] = 0
    name: Name[str] = "var"



**Cases I**: 1D Time Series

* `[360] * [30p] * [30s] = [12]`
* Latent Vector - `[12, Z]`
* Ensemble Predictions - `[12, 20]`

**Case II**: Lat/Lon

* `[200, 200] * [10, 10] = [20, 20]`

**Case III**: Lat/Lon/Time

* `[200, 200, 360] * [10, 10, 30] = [20, 20, 12]`
* 360 Days to 30 Days
* Latent Vector
* Ensemble Predictions

## 1D Temporal Field

We have a 1D time series:



In [5]:
from oceanbench._src.datasets.base import XRDABatcher

In [6]:

x = CoordinateAxis.init_from_limits(x_min=1, x_max=360, dx = 1)
rng = np.random.RandomState(seed=123)
data = rng.randn(*x.data.shape)

@dataclass
class Variable1D:
    data: Data[X, np.ndarray]
    x: Coord[X, np.ndarray] = 0
    name: Name[str] = "var"
    
variable = Variable1D(data=data, x=x.data, name="ssh")
da = asdataarray(variable)
da.x

<xarray.DataArray 'x' (x: 360)>
array([1, 2, 3, ..., 358, 359, 360], dtype=object)
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In [21]:
patches = {"x": 30}
strides = {"x": 5}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('x', 360)])
Patches:        OrderedDict([('x', 30)])
Strides:        OrderedDict([('x', 5)])
Num Batches:    OrderedDict([('x', 67)])
Torch Dataset(size): 67


In [22]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 

all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)

In [23]:
import itertools

items = list(itertools.chain(*[all_batches]))
items_latent = list(itertools.chain(*[all_batches_latent]))

In [10]:
xrda_batches.coord_names

['x']

In [51]:
# create items
all_batches = list(map(lambda x: x, xrda_batches))
items = list(itertools.chain(*[all_batches]))

# CASE I - Weight Matrix | Explicit Label
dims_labels = ["x"]
weight = np.ones([30,])
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

# CASE II - No Weight Matrix | Explicit Label
dims_labels = ["x"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels, weight=weight)
np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

# CASE III - No Weight Matrix | No Label
dims_labels = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items, dims_labels=dims_labels, weight=weight)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)
rec_da



100%|██████████| 67/67 [00:00<00:00, 171.32it/s]


<xarray.DataArray (x: 360)>
array([-1.08563066e+00,  9.97345448e-01,  2.82978505e-01, -1.50629473e+00,
       -5.78600228e-01,  1.65143657e+00, -2.42667913e+00, -4.28912640e-01,
        1.26593626e+00, -8.66740406e-01, -6.78886175e-01, -9.47089717e-02,
        1.49138963e+00, -6.38902009e-01, -4.43981946e-01, -4.34351265e-01,
        2.20592999e+00,  2.18678617e+00,  1.00405395e+00,  3.86186391e-01,
        7.37368584e-01,  1.49073207e+00, -9.35833871e-01,  1.17582905e+00,
       -1.25388062e+00, -6.37751520e-01,  9.07105207e-01, -1.42868066e+00,
       -1.40068725e-01, -8.61754894e-01, -2.55619377e-01, -2.79858899e+00,
       -1.77153313e+00, -6.99877262e-01,  9.27462459e-01, -1.73635677e-01,
        2.84591597e-03,  6.88222706e-01, -8.79536331e-01,  2.83627331e-01,
       -8.05366516e-01, -1.72766948e+00, -3.90899807e-01,  5.73805869e-01,
        3.38589042e-01, -1.18304947e-02,  2.39236522e+00,  4.12912160e-01,
        9.78735983e-01,  2.23814344e+00, -1.29408526e+00, -1.03878820e+00,
        1.74371219e+00, -7.98062742e-01,  2.96832304e-02,  1.06931591e+00,
        8.90706420e-01,  1.75488615e+00,  1.49564409e+00,  1.06939268e+00,
       -7.72708714e-01,  7.94862688e-01,  3.14271986e-01, -1.32626545e+00,
        1.41729903e+00,  8.07236552e-01,  4.54900824e-02, -2.33092055e-01,
       -1.19830120e+00,  1.99524075e-01,  4.68439132e-01, -8.31155002e-01,
        1.16220403e+00, -1.09720302e+00, -2.12310028e+00,  1.03972709e+00,
       -4.03366029e-01, -1.26029581e-01, -8.37516725e-01, -1.60596275e+00,
...
       -1.25118983e+00, -1.18432736e+00, -1.51810801e+00, -4.61187422e-01,
       -3.54908824e-01, -6.82538152e-01, -1.65369785e+00,  1.25333595e+00,
       -1.32907879e+00,  2.78033704e-01, -1.07476664e+00,  6.68316841e-01,
        9.55832362e-01, -8.77613604e-01, -1.92371571e+00,  6.95787311e-01,
        1.87580049e+00,  4.15694535e-01,  1.60544425e-01,  8.19760621e-01,
        7.65054822e-01, -8.28988850e-01, -6.59151316e-01,  6.11123562e-01,
       -1.44013345e-01,  1.31660557e+00, -7.04342127e-01,  7.50609934e-01,
        3.42637986e-01, -1.26437560e-01,  1.17591083e+00,  6.80071533e-01,
       -1.00496721e+00,  6.40218675e-01,  1.37499058e+00, -1.30444691e-01,
       -2.48655856e-01, -6.69647157e-01, -1.36038857e-02,  6.86200678e-01,
       -8.17668319e-01, -1.34635758e+00, -3.75749916e-01, -1.37972498e+00,
        5.23218453e-01, -4.26689774e-01, -1.75540185e+00, -3.48607510e-01,
       -1.92614987e-01,  4.49135602e-01, -1.45363539e-01,  1.86872649e+00,
       -5.18703878e-01, -6.23985492e-02, -1.02910616e-01, -2.82628387e-01,
        1.42425582e-01,  5.41231275e-01,  1.34009874e+00, -1.56925619e+00,
       -5.10342896e-01, -4.47771430e-01,  9.37850296e-01, -3.56663048e-01,
       -1.89517558e+00,  8.77304673e-02, -3.36892307e-02,  1.79751560e-01,
       -1.04016292e+00,  1.71903467e+00, -3.23859781e-01, -1.88296854e-01,
       -9.00008559e-01, -9.31002021e-01, -1.22273695e+00, -3.93310845e-01,
       -9.57581818e-01,  2.05646729e+00, -1.88849235e+00, -1.12833059e+00])
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In [71]:
from einops import rearrange
all_batches = list(map(lambda x: x, xrda_batches))
all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

# CASE I - Weight  | Label (All)
dims_labels_latent = ["x", "z"]
weight = np.ones([30,])
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I - Weight  | Label (All) - switched
dims_labels_latent = ["z", "x"]
weight = np.ones([30,])
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)

# CASE I -  No Weight |  Label (All) - Switched
dims_labels_latent = ["z", "x"]
weight = None
all_batches_latent_flipped = list(map(lambda x: repeat(x, "D Z -> Z D"), all_batches_latent)) 
items_latent_flipped = list(itertools.chain(*[all_batches_latent_flipped]))
rec_da = xrda_batches.reconstruct_from_items(items_latent_flipped, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da)


# CASE I -  No Weight | Half Label (All)
dims_labels_latent = ["x"]
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

# CASE I -  No Weight | No Label (All)
dims_labels_latent = None
weight = None
rec_da = xrda_batches.reconstruct_from_items(items_latent, dims_labels_latent, weight=weight)
np.testing.assert_array_almost_equal(rec_da.isel(v1=0).data, xrda_batches.da)

rec_da

100%|██████████| 67/67 [00:00<00:00, 294.53it/s]


<xarray.DataArray (x: 360, v1: 5)>
array([[-1.08563066, -1.08563066, -1.08563066, -1.08563066, -1.08563066],
       [ 0.99734545,  0.99734545,  0.99734545,  0.99734545,  0.99734545],
       [ 0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ,  0.2829785 ],
       ...,
       [ 2.05646729,  2.05646729,  2.05646729,  2.05646729,  2.05646729],
       [-1.88849235, -1.88849235, -1.88849235, -1.88849235, -1.88849235],
       [-1.12833059, -1.12833059, -1.12833059, -1.12833059, -1.12833059]])
Coordinates:
  * x        (x) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360
Dimensions without coordinates: v1

## Test Case II - 2D Field

In [69]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [70]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50-0.05, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50-0.05, dlat=0.05)

da = SSH2D.init_from_axis(lon=lon_axis, lat=lat_axis)
da = asdataarray(da).to_dataset()
da

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 200)
Coordinates:
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Data variables:
    ssh      (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

In [251]:
patches = {"lat": 40, "lon": 50}
strides = {"lat": 20, "lon": 10}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da.ssh,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('lat', 200), ('lon', 200)])
Patches:        OrderedDict([('lat', 40), ('lon', 50)])
Strides:        OrderedDict([('lat', 20), ('lon', 10)])
Num Batches:    OrderedDict([('lat', 9), ('lon', 16)])
Torch Dataset(size): 144


In [252]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(40, 50)

In [253]:
all_coords[0][["lon"]].coords

Coordinates:
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 42.3 42.35 42.4 42.45

In [258]:
# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,200))
w = 10*np.ones((40,50))
dims_label = ["lat", "y"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w
rec_da = xrda_batches.reconstruct([all_batches], dims_label, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

# np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)


100%|████████████████████████████████████████| 144/144 [00:00<00:00, 540.49it/s]


In [265]:
list(rec_da.coords.keys())
# np.testing.assert_array_equal(rec_da["lat"], xrda_batches.da["lat"])

['lat']

In [223]:
rec_da.coords.dims

('lat', 'v1')

In [205]:
rec_da.min(), rec_da.max()

(<xarray.DataArray ()>
 array(1.),
 <xarray.DataArray ()>
 array(1.))

In [55]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((40,200))
dims_label_latent = ["lat", "lon", "z"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N", N=5), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

np.testing.assert_array_almost_equal(rec_da.isel(z=0).data, xrda_batches.da.data)

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 185.41it/s]


## Test Case III - 2D+T Field

In [38]:
from oceanbench._src.utils.custom_dtypes import (
    LongitudeAxis, LatitudeAxis, 
    TimeAxis, SSH2D, SSH2DT
)

In [39]:
lon_axis = LongitudeAxis.init_from_limits(lon_min=40, lon_max=50-0.05, dlon=0.05)
lat_axis = LatitudeAxis.init_from_limits(lat_min=-60, lat_max=-50-0.05, dlat=0.05)
time_axis = TimeAxis.init_from_limits("2012-01-01", "2012-01-30", "1D")

da = SSH2DT.init_from_axis(lon=lon_axis, lat=lat_axis, time=time_axis)
da = asdataarray(da).to_dataset()
da

<xarray.Dataset>
Dimensions:  (time: 30, lat: 200, lon: 200)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-01-30
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Data variables:
    ssh      (time, lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [40]:
# patches = {"time": 10, "lat": 40, "lon": 20, }
patches = {"lon": 20, "time": 10, "lat": 40, }
strides = {"time": 10, "lat": 40, "lon": 20, }
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da.ssh,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

# assert ds[0].shape == (len(var.data),)
# np.testing.assert_array_equal(ds[0], np.arange(0, len(var.x), 1))
print(xrda_batches)
print(f"Torch Dataset(size): {len(xrda_batches)}")
# print(f"Batch (size): {ds[0].shape}")

XArray Patcher
DataArray size: OrderedDict([('time', 30), ('lat', 200), ('lon', 200)])
Patches:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Strides:        OrderedDict([('time', 10), ('lat', 40), ('lon', 20)])
Num Batches:    OrderedDict([('time', 3), ('lat', 5), ('lon', 10)])
Torch Dataset(size): 150


In [41]:
import functools as ft
import itertools as it
from einops import repeat

all_batches = list(map(lambda x: x, xrda_batches))


all_coords = xrda_batches.get_coords()

assert len(all_batches) == len(all_coords)
all_batches[0].shape

(10, 40, 20)

In [42]:
from oceanbench._src.datasets.utils import reconstruct_from_items

# reconstruct_from_items(items_latent, ["x","z"], coords)
w = np.ones((10,40,20))
dims_label = ["time", "lat", "lon"]
items = list(itertools.chain(*[all_batches]))
# rec_da = xrda_batches.reconstruct(all_batches, dims_label, weight=w)
rec_da = reconstruct_from_items(items, dims_label, xrda_batches, weight=w)
# rec_da = reconstruct_from_items(items_latent, dims_label_latent, ds, weight=w)

np.testing.assert_array_almost_equal(rec_da.data, xrda_batches.da)

100%|████████████████████████████████████████| 150/150 [00:00<00:00, 220.63it/s]


In [43]:
from oceanbench._src.datasets.utils import reconstruct_from_items

w = np.ones((10,40,20))
dims_label_latent = ["time", "lat", "lon", "z", "d"]

all_batches_latent = list(map(lambda x: repeat(x, "... -> ... N M", N=5, M=10), all_batches)) 
items_latent = list(itertools.chain(*[all_batches_latent]))

rec_da = reconstruct_from_items(items_latent, dims_label_latent, xrda_batches, weight=w)

100%|█████████████████████████████████████████| 150/150 [00:03<00:00, 43.59it/s]


In [44]:
np.testing.assert_array_almost_equal(rec_da.isel(z=0, d=0).data, xrda_batches.da.data)

In [45]:
rec_da

<xarray.DataArray (time: 30, lat: 200, lon: 200, z: 5, d: 10)>
array([[[[[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         ...,

...

         ...,

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]],

         [[1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.],
          [1., 1., 1., ..., 1., 1., 1.]]]]])
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-01-30
  * lat      (lat) float32 -60.0 -59.95 -59.9 -59.85 ... -50.15 -50.1 -50.05
  * lon      (lon) float32 40.0 40.05 40.1 40.15 40.2 ... 49.8 49.85 49.9 49.95
Dimensions without coordinates: z, d